In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold,cross_validate, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

In [3]:
#merging all the required datasets - metabolomics dataset, drug response dataset and the metadata of the cell lines

data = pd.read_excel("D:\\GaTech\\Fall Sem\\BIOL 8901\\Metabolomic Project\\metabolomic_data.xlsx", sheet_name="1-clean data")
data.rename(columns = {'Unnamed: 0': 'ID'}, inplace = True)
last_column = data.iloc[:,-1].name
sample = pd.read_csv("D:\\GaTech\\Fall Sem\\BIOL 8901\\sample_info.csv")
sample.rename(columns = {'CCLE_Name':'ID'}, inplace=True)
merged_data = data.merge(sample, on='ID')
drug = pd.read_csv('D:\\GaTech\\Fall Sem\\BIOL 8901\\sanger-dose-response.csv')
drug.rename(columns={'ARXSPAN_ID':'DepMap_ID'}, inplace = True)
working_data = merged_data.merge(drug, on='DepMap_ID')

In [4]:
#since our data has a lot of NaN's in them, we can fill them using 0 (for now, just for a workaround)

working_data.fillna(0, inplace=True)

In [5]:
X1 = working_data.loc[working_data['DRUG_NAME'] == 'GEMCITABINE']

In [6]:
'''
reset the index as once we take a subset of the main working_dataset, the indexes will get mixed up.
thus the indices need to be reset before we start working on the model
'''

X1.reset_index(inplace=True)

In [7]:
X1.iloc[:, 252:271].head(2)

,DATASET,COSMIC_ID,DRUG_ID,MIN_CONC,MAX_CONC,RMSE_PUBLISHED,Z_SCORE_PUBLISHED,IC50_PUBLISHED,AUC_PUBLISHED,DRUG_NAME,BROAD_ID,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2
0,GDSC1,907295,135,0.0040,1.024,0.069992,0.786393,0.756651,0.832569,GEMCITABINE,"BRD-K15108141, BRD-K67681739, BRD-K11238904",0.971751,0.195748,-1.961257,0.617560,0.864834,0.0,0.001675,0.911867
1,GDSC2,907295,1190,0.0001,10.000,0.184224,0.536873,5.526728,0.880510,GEMCITABINE,"BRD-K15108141, BRD-K67681739, BRD-K11238904",1.290283,0.361760,-0.087955,0.269134,0.833094,0.0,0.006039,0.458854


In [9]:
#wherever there is any duplicate within a cell line's IC50 values, take the cell line that originates from the GDSC2 phase 
X_ = X1[~X1.duplicated(['DepMap_ID'], keep=False) | X1['DATASET'].eq('GDSC2')]

In [10]:
X_.reset_index(inplace=True)

In [11]:
X_.shape

(606, 272)

In [12]:
#drop the metadata

X_ = X_.select_dtypes('float64')
X_.shape

(606, 244)

In [14]:
#scale the dataframe, at a quick glance the metabolite profiles are not scaled thus scaling of the dataframe is necessary

X_min = X_.min()
X_max = X_.max()
X_range = (X_max-X_min)
X_scaled = (X_-X_min)/(X_range)
X_scaled.head(5)

,2-aminoadipate,3-phosphoglycerate,alpha-glycerophosphate,4-pyridoxate,aconitate,adenine,adipate,alpha-ketoglutarate,AMP,citrate,...,IC50_PUBLISHED,AUC_PUBLISHED,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2
0,0.395300,0.621262,0.546764,0.459783,0.417356,0.380599,0.587679,0.438061,0.405262,0.587155,...,0.001093,0.881922,0.004116,1.266699e-272,0.995922,0.999999,0.836094,0.740036,0.096037,0.459348
1,0.215830,0.454296,0.314096,0.491937,0.544737,0.363184,0.521658,0.473832,0.353421,0.699079,...,0.010044,0.945487,0.002992,6.494907e-272,0.771003,1.000000,0.924929,0.740036,0.011433,0.591835
2,0.319421,0.371580,0.441413,0.392502,0.484120,0.383373,0.526204,0.507683,0.347641,0.558086,...,0.000016,0.856171,0.002960,1.668918e-273,0.958298,0.999999,0.848425,0.740036,0.029455,0.867611
3,0.321303,0.624052,0.669008,0.302954,0.412029,0.422752,0.535574,0.194364,0.749331,0.502986,...,0.000018,0.588445,0.003132,1.760479e-273,0.838750,0.999999,0.704726,0.443158,0.027500,0.979280
4,0.436124,0.642319,0.613914,0.242486,0.736116,0.445114,0.550754,0.611088,0.402429,0.767874,...,0.000008,0.514529,0.003135,5.575603e-274,0.923040,0.999999,0.685647,0.420507,0.022121,0.973502


In [15]:
#calculate the mean of the IC50_PUBLISHED

drug_mean = X_scaled['IC50_PUBLISHED'].mean()
print(drug_mean)

0.011854742117440796


In [16]:
#create labels

ic, labels = X_scaled['IC50_PUBLISHED'], []
for i in range(len(ic)):
    #if the IC50_PUBLISHED value is greater than the mean, add responsive label to the label list
    if ic[i] > drug_mean:
        labels.append('R')
    #if the IC50_PUBLISHED value is less than the mean, add non responsive label to the label list
    elif ic[i] < drug_mean:
        labels.append('NR')
        
y = pd.Series(labels)

In [17]:
X = X_scaled.iloc[:, :225]
X.head(1)

,2-aminoadipate,3-phosphoglycerate,alpha-glycerophosphate,4-pyridoxate,aconitate,adenine,adipate,alpha-ketoglutarate,AMP,citrate,...,C56:8 TAG,C56:7 TAG,C56:6 TAG,C56:5 TAG,C56:4 TAG,C56:3 TAG,C56:2 TAG,C58:8 TAG,C58:7 TAG,C58:6 TAG
0,0.3953,0.621262,0.546764,0.459783,0.417356,0.380599,0.587679,0.438061,0.405262,0.587155,...,0.683987,0.527663,0.679875,0.599994,0.609317,0.650514,0.463536,0.569028,0.51641,0.512918


In [18]:
#calculate the correlation matrix of the metabolite dataframe
#choose the upper triangle of the correlation matrix
#create a list of features where the correlation value is >0.90
#this list contains the highly correlated features, which will be removed from the dataset

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column]>0.90)]

In [19]:
#drop the list of highly correlated features computed above

X.drop(to_drop, axis=1, inplace=True)

In [20]:
len(to_drop)

22

In [21]:
X.shape, y.shape

((606, 203), (606,))

In [22]:
#run the RFECV model with estimator being Random Forest and StratifiedKFold cross validation with 5 folds.

rfecv = RFECV(estimator = RandomForestClassifier(random_state=101), step=1, cv=StratifiedKFold(5), scoring='accuracy')

In [23]:
#fit the X,y to the RFECV model

rfecv.fit(X, y)

RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
      estimator=RandomForestClassifier(random_state=101), scoring='accuracy')

In [24]:
#choose the features that are selected by the RFECV model

selected_features = rfecv.get_support(1)

In [25]:
#select a subset dataframe that contains only the "optimal" metabolic features returned from the RFECV model

X3 = X[X.columns[selected_features]]
X3.shape

(606, 54)

In [26]:
#using classification_report metrics, run a prediction model using StratifiedKFold cross_validation with k=5 folds
#model being used as the classifier is Random Forest

kf3 = StratifiedKFold(n_splits = 5, shuffle=False)
model=RandomForestClassifier()
i=1
dfs = []
for train_index, test_index in kf3.split(X3, y):
    #select train and test datasets from X and y
    X_train, X_test = X3.iloc[train_index], X3.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #train the model
    model.fit(X_train, y_train)
    #predict the test dataset
    predicted = model.predict(X_test)
    #print the classification score report
    report = classification_report(y_test, predicted, output_dict = True)
    df = pd.DataFrame(report).transpose() 
    dfs.append(df)
    i+=1
results_df = pd.concat(dfs)
results_df.to_csv("GEMCITABINE.tsv", sep="\t")

In [28]:
results_df

,precision,recall,f1-score,support
NR,0.885246,1.000000,0.939130,108.000000
R,0.000000,0.000000,0.000000,14.000000
accuracy,0.885246,0.885246,0.885246,0.885246
macro avg,0.442623,0.500000,0.469565,122.000000
weighted avg,0.783660,0.885246,0.831361,122.000000
NR,0.892562,1.000000,0.943231,108.000000
R,0.000000,0.000000,0.000000,13.000000
accuracy,0.892562,0.892562,0.892562,0.892562
macro avg,0.446281,0.500000,0.471616,121.000000
weighted avg,0.796667,0.892562,0.841893,121.000000
